In [1]:
import os
os.chdir('../..')
!ls

product_impacts


In [2]:
import pandas as pd
import numpy as np
import re
import unicodedata
from skimpy import skim
from tqdm.notebook import tqdm

from product_impacts.product_cat.run_model import get_store

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
tqdm.pandas()

#### Food db data

In [3]:
fdb = pd.read_csv('../../SFS/NDNS UK/predictions/all_predictions_15Oct2024.csv')

products1 = pd.read_csv('../../SFS/all_fooddb_data_extracts/Extract_2019/foodDB_Raw/products.csv', usecols= [
    'product_id', 'product_list_name', 'product_name', 'ingredients_text',  
    'energy_per_100', 'fat_per_100', 'saturates_per_100', 'salt_per_100', 
    'sugar_per_100', 'carbohydrate_per_100', 'protein_per_100', 'fibre_per_100',
    'serving', 'serving_value', 'serving_unit']).rename(columns={
    'serving': 'serving_size', 
    'serving_value': 'serving_size_value', 
    'serving_unit': 'serving_size_unit'})
products2 = pd.read_csv('../../SFS/all_fooddb_data_extracts/Extract_2021/foodDB_dat/products.csv', usecols= [
    'product_id', 'product_list_name', 'product_name', 'ingredients_text', 
    'energy_per_100', 'fat_per_100', 'saturates_per_100', 'salt_per_100', 
    'sugar_per_100', 'carbohydrate_per_100', 'protein_per_100', 'fibre_per_100',
    'serving', 'serving_value', 'serving_unit']).rename(columns={
    'serving': 'serving_size', 
    'serving_value': 'serving_size_value', 
    'serving_unit': 'serving_size_unit'})
products3 = pd.read_csv('../../SFS/all_fooddb_data_extracts/Extract_2022/May_2022_Extract/products.csv',  usecols= [
    'foodDB_product_id', 'product_list_name', 'product_name', 'ingredients_list', 
    'Energy', 'Fat', 'Saturates', 'Salt', 'Sugar', 'Carbohydrate', 'Fibre', 'Protein',
    'serving_size', 'serving_size_value', 'serving_size_unit']).rename(columns={
    'foodDB_product_id': 'product_id',
    'ingredients_list': 'ingredients_text',
    'Energy': 'energy_per_100',
    'Fat': 'fat_per_100', 
    'Saturates': 'saturates_per_100', 
    'Salt': 'salt_per_100', 
    'Sugar': 'sugar_per_100', 
    'Carbohydrate': 'carbohydrate_per_100', 
    'Protein': 'protein_per_100', 
    'Fibre': 'fibre_per_100'})
products = pd.concat([products1, products2, products3], axis=0, ignore_index=True)

products = products.drop_duplicates(['product_id']).reset_index(drop=True)
products.loc[(products['product_name'].notnull()) 
             & (products['product_name'].str.len()>products['product_list_name'].str.len()),
             'product_list_name'] = products[
    (products['product_name'].notnull()) 
    & (products['product_name'].str.len()>products['product_list_name'].str.len())]['product_name']
products['ingredients_text'] = products['ingredients_text'].fillna('')

fdb = products.merge(fdb[['product_id', 'store', 'parentcategory_pred', 'mainfoodgroup_pred', 'subfoodgroup_pred']])

/Users/shruti/anaconda3/envs/ct_map/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/shruti/anaconda3/envs/ct_map/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (37,38,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
fdb.shape

(262711, 19)

In [5]:
fdb = fdb[~fdb['parentcategory_pred'].isin(['Alcoholic Beverages', 'Not Food', 
                                            # 'Artificial Sweeteners'
                                           ])]

In [6]:
# replacing grams with % (doing this separately here as the cleaning function function removes *)
exclusion_pattern = r'(?!.*egg\*\*)(?!.*veg\*\*)(?!.*ing\*\*)'
pattern = rf'^{exclusion_pattern}.*g\*\*.*$'
combined_condition = fdb['ingredients_text'].str.contains(pattern, case=False, regex=True)
fdb.loc[combined_condition, 'ingredients_text'] = fdb.loc[combined_condition, 
                                                          'ingredients_text'].str.replace('g**', '%', regex=False)

In [7]:
fdb.shape

(242475, 19)

In [8]:
fdb['parentcategory_pred'].value_counts()

Cereals and Cereal Products             52426
Miscellaneous                           27598
Meat and Meat Products                  26837
Non-Alcoholic Beverages                 25994
Sugar, Preserves and Confectionery      25822
Vegetables, Potatoes                    23757
Milk and Milk Products                  23239
Fish and Fish Dishes                     9302
Fruit                                    7343
Savoury Snacks                           6460
Nuts and Seeds                           5057
Fat Spreads                              3811
Commercial Toddlers Foods and Drinks     2488
Eggs and Egg Dishes                      1315
Dietary Supplements                       622
Artificial Sweeteners                     404
Name: parentcategory_pred, dtype: int64

#### Open food facts data

In [9]:
# it is okay to just go with the neaural network oredictions and impose a 0.8 threshold (maybe this is too conservative, can rethink)
off = pd.read_csv('../../SFS/openfoodfacts/all/openfoodfacts_lang.csv')

off = off[['product_id', 'product_name', 'ingredients_text',
           'product_name_en', 'ingredients_text_en', 'countries_en', 'stores',
           'energy_100g', 'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 
           'fiber_100g', 'proteins_100g', 'sugars_100g', 'salt_100g', 
           'serving_size', 'serving_quantity']].rename(columns={
    'energy_100g': 'energy_per_100', 
    'fat_100g': 'fat_per_100', 
    'saturated-fat_100g': 'saturates_per_100', 
    'salt_100g': 'salt_per_100', 
    'sugars_100g': 'sugar_per_100',
    'carbohydrates_100g': 'carbohydrate_per_100', 
    'proteins_100g': 'protein_per_100', 
    'fiber_100g': 'fibre_per_100'})

preds = []

for prefix in ['eng/', 'non_eng/']:
    pred = pd.read_csv(f'../../SFS/openfoodfacts/all/predictions/{prefix}predictions_nn.csv')

    # pred = pred[(pred['parentcategory_prob']>0.8)
    #                   & ((pred['subfoodgroup_prob'].isna()) | (pred['subfoodgroup_prob']>0.8))]
    
    preds.append(pred)

preds = pd.concat(preds, axis=0, ignore_index=True)
off = off.merge(preds)

/Users/shruti/anaconda3/envs/ct_map/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (15,16,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
off[off['ingredients_text_en'].notnull()].shape

(960365, 22)

In [11]:
off = off[~off['parentcategory'].isin(['Alcoholic Beverages', 'Not Food', 
                                       # 'Artificial Sweeteners'
                                      ])]

In [12]:
off['ingredients_text_en'] = off['ingredients_text_en'].fillna('')

In [13]:
off.shape

(1213083, 22)

In [14]:
off[off['ingredients_text_en']!=''].shape

(932678, 22)

In [15]:
off[off['ingredients_text_en']!='']['countries_en'].value_counts()

United States           299563
France                  273609
Germany                  71872
Spain                    35922
United Kingdom           34127
Switzerland              24609
Italy                    22646
Belgium                  19255
World                    18443
Canada                   12990
Netherlands              11708
Australia                 8909
Poland                    6917
Ireland                   5555
Mexico                    5418
Austria                   4977
Portugal                  4577
Bulgaria                  4576
Sweden                    4177
Czech Republic            3936
Norway                    3612
Romania                   3461
Finland                   3376
New Zealand               3096
Argentina                 3066
Thailand                  2844
Russia                    2672
Bolivia                   2612
Brazil                    2408
India                     2204
Hungary                   1947
Croatia                   1658
Luxembou

In [16]:
off['parentcategory'].value_counts()

Cereals and Cereal Products             254065
Sugar, Preserves and Confectionery      153977
Vegetables, Potatoes                    141228
Milk and Milk Products                  138615
Non-Alcoholic Beverages                 116132
Meat and Meat Products                  115478
Miscellaneous                           106677
Fruit                                    47197
Nuts and Seeds                           39211
Fish and Fish Dishes                     37821
Savoury Snacks                           31372
Fat Spreads                              19052
Eggs and Egg Dishes                       5294
Dietary Supplements                       3964
Commercial Toddlers Foods and Drinks      1642
Artificial Sweeteners                     1358
Name: parentcategory, dtype: int64

In [17]:
# off[off['countries_en'].isin(['United States', 'France', 'United Kingdom', 'Germany',
#                               'Spain', 'Switzerland', 'Italy', 'Canada', 'Belgium',
#                               'Australia', 'Ireland'])].groupby(
#     ['countries_en', 'parentcategory']).count()[['product_id']].reset_index().pivot(
#     index='countries_en', columns='parentcategory', values='product_id')

#### Generic cleaning funciton - can be applied to both food db and open food facts

In [18]:
def clean_ingredients(text, is_fdb=True):
    
    # Mapping for special characters not handled by unicodedata
    special_replacements = {
        'ß': 'ss',
        'Æ': 'ae',
        'æ': 'ae',
        'Œ': 'oe',
        'œ': 'oe',
        'Ø': 'o',
        'ø': 'o',
        'Đ': 'd',
        'đ': 'd',
        'Þ': 'th',
        'þ': 'th',
        # Add more mappings as needed
    }
    
    # Removing unwanted characters and words
    text = text.lower()
    text = text.replace("*", " ").replace("_", " ").replace("’", "'").replace("?", " ").replace("=", " ").replace("^", " ")
    text = re.sub(' +', ' ', text)
    text = text.replace(";", ",").replace(",,", ",").replace(", ,", ",")
    text = text.replace(": (", "(")
    text = text.replace("“", "").replace("”", "")
    text = text.replace("& quot;", "")  # remove html encoded quotes
    text = text.replace("â€\xa0", "")
    text = text.replace("\u200b", "")
    text = text.replace("&", "and")      # replace ampersand
    text = text.replace('ingredients:', '')
    text = text.replace('ingredients list:', '')
    text = re.sub(r'(?<!\s)\(', ' (', text)
    
    # removing diacritics and replacing special Latin characters
    for char, replacement in special_replacements.items():
        text = text.replace(char, replacement)
    # Normalize the string to NFD (Normalization Form Decomposition)
    text = unicodedata.normalize('NFD', text)
    # Remove combining diacritical marks
    text = re.sub(r'[\u0300-\u036f]', '', text)
    
    # Converting brackets to parentheses
    text = re.sub("\[|\{","(", text)
    text = re.sub("\]|\}",")", text)

    # Replacing "per cent" with "%"
    text = re.sub(r"per(\s)?cent", "%", text)
    text = text.replace(" %", "%")
    text = text.replace("%", "% ")
    text = text.replace("( ", "(").replace(" )", ")")
    text = re.sub(' +', ' ', text)
    text = text.replace("%%", "%")
    
    # Fix "(10% brown" → "(10%) brown"
    text = re.sub(r'\((\d+(?:\.\d+)?)%(\s)', r'(\1%)\2', text)
    # Fix "grapes 10%)" → "grapes (10%)" 
    text = re.sub(r'\s+([+-]?\d+(?:\.\d+)?)%\)', r' (\1%)', text)
    # Fix "10%" → "(10%)"
    text = re.sub(r'(?<!\()(\d+(?:\.\d+)?)%(?!\))', r'(\1%)', text)
    
    # Removing specific allergen advice text and other labels
    if is_fdb:
        text = text.replace("allergy Advice: for allergens see highlighted ingredients", "")
        text = text.replace("for allergens, see undelined ingredients in bold.", "")
    text = re.sub(r'allerg.*', '', text)
    text = re.sub(r'it may contain.*', '', text)
    text = re.sub(r'may contain.*', '', text)
    text = re.sub(r'our method.*', '', text)
    text = re.sub(r'www.*', '', text)
    text = re.sub(r'dietary advice.*', '', text)
    text = re.sub(r'not suitable for.*', '', text)
    text = re.sub(r'suitable for.*', '', text)
    text = re.sub(r'this product is made in.*', '', text)
    text = re.sub(r'made in.*', '', text)
    text = re.sub(r'free from.*', '', text)
    text = re.sub(r'rainforest alliance.*', '', text)
    text = re.sub(r'organic ingredients certified.*', '', text)
    text = re.sub(r'certified organic ingredients.*', '', text)
    text = re.sub(r'from sustainably grown.*', '', text)
    text = text.replace('sustainably grown', '')
    text = re.sub(r'†from.*', '', text)
    text = text.replace("†", " ").strip()
    text = re.sub(' +', ' ', text)

    # removing empty brackets
    text = text.replace("()"," ").strip()
    text = re.sub(' +', ' ', text)
    
    return text

In [19]:
fdb['ingredients_text'] = fdb.progress_apply(lambda row: clean_ingredients(row['ingredients_text'], True), axis=1)

  0%|          | 0/242475 [00:00<?, ?it/s]

In [20]:
off['ingredients_text_en'] = off.progress_apply(lambda row: clean_ingredients(row['ingredients_text_en'], False), axis=1)

  0%|          | 0/1213083 [00:00<?, ?it/s]

#### cleaning functions only for open food facts

In [21]:
def clean_ingredients2(text):
    
    # handling ' - ' separaters 
    if ',' not in text:
        text = text.replace(" - ", ", ")
        
    # removing origin: ...
    text = re.sub("\(origin:[^)]*\)", "", text)
    text = re.sub("\(country of origin:[^)]*\)", "", text)
    text = re.sub("origin:[^.]*\.", "", text)
    
    text = re.sub(r'although every care.*', '', text)
    
    return text

In [22]:
off['ingredients_text_en'] = off.progress_apply(lambda row: clean_ingredients2(row['ingredients_text_en']), axis=1)

  0%|          | 0/1213083 [00:00<?, ?it/s]

In [23]:
off = off[off['countries_en']!='World']

In [24]:
# off = off[off['countries_en'].isin(['United States', 'France', 'United Kingdom', 'Germany',
#                                     'Spain', 'Switzerland', 'Italy', 'Canada', 'Belgium',
#                                     'Australia', 'Ireland'])].reset_index(drop=True)

In [25]:
# 2 rows have large and weird strings, removing those
off.loc[(off['ingredients_text_en'].str.contains('helado sabor a vainilla 55,(5%)', regex=False)), 
        'ingredients_text_en'] = ''

In [26]:
allowed_pattern = r'^[A-Za-z0-9.\(\),% ]+$' # allowing only alphanumeric, periods, percentages, parantheses, commas
off1 = off[(off['ingredients_text_en'].str.match(allowed_pattern))
           | (off['ingredients_text_en']=='')
          ]

off2 = off[(~off['ingredients_text_en'].str.match(allowed_pattern))
           & (off['ingredients_text_en']!='')
          ]

In [27]:
off1.shape

(728336, 22)

In [28]:
off2.shape # figure out if any of these are usable

(449560, 22)

In [29]:
def correct_structure(s, max_iterations=10):
    """
    Corrects the structure of the input string by ensuring that percentages
    are placed immediately after the relevant words within each segment.

    Args:
        s (str): The input string to be corrected.

    Returns:
        str: The corrected string with properly structured segments.
    """
    s = re.sub(' +', ' ', s)
    s = re.sub(r'\.$', '', s)
    s = re.sub(r'\,$', '', s)

    # Pattern 1: (percentage) word => word (percentage)
    pattern1 = re.compile(r'\((\d+(\.\d+)?)%\)\s+([^(),]+)')
    replacement1 = r'\3 (\1%)'

    # Pattern 2: word (description) (percentage) => word (percentage) (description)
    pattern2 = re.compile(r'([^(),]+)\s*\(([^()]+)\)\s*\((\d+(\.\d+)?)%\)')
    replacement2 = r'\1 (\3%) (\2)'

    iterations = 0
    previous_s = None

    while previous_s != s and iterations < max_iterations:
        previous_s = s
        s = pattern1.sub(replacement1, s)
        s = pattern2.sub(replacement2, s)
        iterations += 1

    if iterations >= max_iterations:
        # Optionally, log the problematic string for debugging
        # print(f"Max iterations reached. Replacing string with empty string: {s}")
        return ''
    
    # Clean up any extra spaces resulting from substitutions
    s = re.sub(r'\s+', ' ', s)
    
    s = s.replace('(', ' (')
    s = re.sub(' +', ' ', s)

    return s.strip()

In [30]:
off1['ingredients_text_en'] = off1['ingredients_text_en'].progress_apply(correct_structure)

  0%|          | 0/728336 [00:00<?, ?it/s]

/Users/shruti/anaconda3/envs/ct_map/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
def clean_ingredients3(text):
    # remove may contain, traces etc
    
    text = re.sub(r'it can contain.*', '', text)
    text = re.sub(r'can contain.*', '', text)
    text = re.sub(r'it does not contain.*', '', text)
    text = re.sub(r'this product coes not contain.*', '', text)
    text = re.sub(r'this chocolate coes not contain.*', '', text)
    text = re.sub(r'does not contain.*', '', text)
    text = re.sub(r'possible traces.*', '', text)
    text = re.sub(r'keep refrigerated.*', '', text)
    
    text = re.sub(r'contains or less of the following:', '', text)
    text = re.sub(r'contains less than of the following:', '', text)
    text = re.sub(r'contains two% or less of the following:', '', text)
    
    text = re.sub(r'contains or less of :', '', text)
    text = re.sub(r'contains less than of :', '', text)
    text = re.sub(r'contains two% or less of :', '', text)
    
    text = re.sub(r'contains or less of the following', '', text)
    text = re.sub(r'contains less than of the following', '', text)
    text = re.sub(r'contains two% or less of the following', '', text)
    
    text = re.sub(r'contains or less of', '', text)
    text = re.sub(r'contains less than of', '', text)
    text = re.sub(r'contains two% or less of', '', text)
    
    text = re.sub(r'contains or less', '', text)
    text = re.sub(r'contains less than', '', text)
    text = re.sub(r'contains two% or less', '', text)
    
    text = re.sub(r'\.$', '', text)
    text = re.sub(r'\,$', '', text)
    text = re.sub(r'(?<!\d)\.', ',', text)
    
    return text.strip()

In [32]:
off1['ingredients_text_en'] = off1['ingredients_text_en'].progress_apply(clean_ingredients3)

  0%|          | 0/728336 [00:00<?, ?it/s]

/Users/shruti/anaconda3/envs/ct_map/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
off1.shape

(728336, 22)

In [34]:
off1[off['ingredients_text_en']!=''].shape

/Users/shruti/anaconda3/envs/ct_map/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(464314, 22)

In [35]:
allowed_pattern = r'^[A-Za-z0-9.\(\),% -]+$' # allowing only alphanumeric, periods, percentages, parantheses, commas, hyphen
off3 = off2[(off2['ingredients_text_en'].str.match(allowed_pattern))]

off4 = off2[(~off2['ingredients_text_en'].str.match(allowed_pattern))]

In [36]:
off2.shape

(449560, 22)

In [37]:
off3.shape

(79097, 22)

In [38]:
off4.shape

(370463, 22)

In [39]:
off3 = off3[(off3['ingredients_text_en'].str.contains('-', regex=False)) 
            & (~off3['ingredients_text_en'].str.contains(' - ', regex=False))]
off3.shape

(61001, 22)

In [40]:
off3['ingredients_text_en'] = off3['ingredients_text_en'].progress_apply(correct_structure)

  0%|          | 0/61001 [00:00<?, ?it/s]

In [41]:
off3['ingredients_text_en'] = off3['ingredients_text_en'].progress_apply(clean_ingredients3)

  0%|          | 0/61001 [00:00<?, ?it/s]

In [42]:
allowed_pattern = r'^[A-Za-z0-9.\(\),%: ]+$' # allowing only alphanumeric, periods, percentages, parantheses, commas, colon
off5 = off4[(off4['ingredients_text_en'].str.match(allowed_pattern))]

off6 = off4[(~off4['ingredients_text_en'].str.match(allowed_pattern))]

In [43]:
off5.shape

(171143, 22)

In [44]:
off6.shape

(199320, 22)

In [45]:
off5 = off5[(off5['ingredients_text_en'].str.contains('preservative:', regex=False))
             | (off5['ingredients_text_en'].str.contains('preservatives:', regex=False))
             | (off5['ingredients_text_en'].str.contains('conservative:', regex=False))
             | (off5['ingredients_text_en'].str.contains('conservatives:', regex=False))
             | (off5['ingredients_text_en'].str.contains('acidifying:', regex=False))
             | (off5['ingredients_text_en'].str.contains('acidity:', regex=False))
             | (off5['ingredients_text_en'].str.contains('acidic:', regex=False))
             | (off5['ingredients_text_en'].str.contains('acid:', regex=False))
             | (off5['ingredients_text_en'].str.contains('color:', regex=False))
             | (off5['ingredients_text_en'].str.contains('colors:', regex=False))
             | (off5['ingredients_text_en'].str.contains('colour:', regex=False))
             | (off5['ingredients_text_en'].str.contains('coloring:', regex=False))
             | (off5['ingredients_text_en'].str.contains('regulator:', regex=False))
             | (off5['ingredients_text_en'].str.contains('regulators:', regex=False))
             | (off5['ingredients_text_en'].str.contains('corrector:', regex=False))
             | (off5['ingredients_text_en'].str.contains('emulgator:', regex=False))
             | (off5['ingredients_text_en'].str.contains('emulsifying:', regex=False))
             | (off5['ingredients_text_en'].str.contains('antioxidant:', regex=False))
             | (off5['ingredients_text_en'].str.contains('fier:', regex=False))
             | (off5['ingredients_text_en'].str.contains('fiers:', regex=False))
             | (off5['ingredients_text_en'].str.contains('zer:', regex=False))
             | (off5['ingredients_text_en'].str.contains('less:', regex=False))
             | (off5['ingredients_text_en'].str.contains('following:', regex=False))
             | (off5['ingredients_text_en'].str.contains('of:', regex=False))
             | (off5['ingredients_text_en'].str.contains('thickening:', regex=False))
             | (off5['ingredients_text_en'].str.contains('contains:', regex=False))
             | (off5['ingredients_text_en'].str.contains('cultures:', regex=False))
             | (off5['ingredients_text_en'].str.contains('agent:', regex=False))]

In [46]:
off5.shape

(123143, 22)

In [47]:
off5['ingredients_text_en'] = off5['ingredients_text_en'].progress_apply(correct_structure)

  0%|          | 0/123143 [00:00<?, ?it/s]

In [48]:
off5['ingredients_text_en'] = off5['ingredients_text_en'].progress_apply(clean_ingredients3)

  0%|          | 0/123143 [00:00<?, ?it/s]

In [49]:
off_clean = pd.concat([off1, off3, off5], axis=0, ignore_index=True)

In [50]:
def clean_ingredients4(text):
    # remove may contain, traces etc
    
    text = re.sub(r'product from organic farming.*', '', text)
    text = re.sub(r'products from organic farming.*', '', text)
    text = re.sub(r'all the ingredients come from organic farmin.*', '', text)
    text = re.sub(r'agricultural ingredient from organic farming.*', '', text)
    text = re.sub(r'(ingredients from organic farming).*', '', text)
    text = re.sub(r'ingredient from organic farming.*', '', text)
    text = re.sub(r'ingredients from organic farming.*', '', text)
    text = re.sub(r'from organic farming.*', '', text)
    
    text = text.replace('origin france', '')
    text = text.replace('origin, france', '')
    text = text.replace('developed from eu origin', '')
    text = text.replace('eu origin, asia', '')
    text = text.replace('eu original', '')
    text = text.replace('eu origin', '')
    text = text.replace('origin eu', '')
    text = text.replace('origin spain', '')
    text = text.replace('brittany origin', '')
    text = text.replace('origin of the netherlands and france', '')
    text = text.replace('zygochlams patagonjca, argentinian origin', '')
    text = text.replace('ingredients origin france (100%)', '')
    text = text.replace('developed from france origin france', '')
    text = text.replace('this product is made from eu original pork', '')
    text = text.replace('origin england or france', '')
    text = text.replace('ingredients of natural origin', '')
    text = text.replace('france', '')
    
    text = text.replace(':', '')
    text = re.sub(' +', ' ', text)
    text = text.strip()
    text = re.sub(r'\.$', '', text)
    text = re.sub(r'\,$', '', text)
    text = text.replace('()', '')
    
    return text.strip()

In [51]:
off_clean['ingredients_text_en'] = off_clean['ingredients_text_en'].progress_apply(clean_ingredients4)

  0%|          | 0/912480 [00:00<?, ?it/s]

In [52]:
off_clean.shape

(912480, 22)

In [53]:
off_clean['countries_en'].value_counts()

United States           315531
France                  218294
Germany                  60431
United Kingdom           52573
Spain                    33780
Canada                   24811
Switzerland              22955
Australia                21486
Italy                    20986
Belgium                  17834
Ireland                  15719
Netherlands               9819
Poland                    5971
New Zealand               4779
Sweden                    4263
Mexico                    4173
Austria                   4165
Portugal                  4144
Finland                   3533
Norway                    3465
Czech Republic            3241
Bulgaria                  2960
India                     2945
Brazil                    2891
Romania                   2598
Bolivia                   2509
Argentina                 2323
Thailand                  2318
South Africa              2153
Singapore                 2107
Saudi Arabia              1918
Japan                     1915
Luxembou

In [54]:
off_clean[off_clean['ingredients_text_en']!=''].shape

(641839, 22)

In [55]:
off_clean[off_clean['ingredients_text_en']!='']['countries_en'].value_counts()

United States           222524
France                  178596
Germany                  50009
Spain                    25502
United Kingdom           25336
Switzerland              18095
Italy                    14997
Belgium                  12923
Canada                    8801
Netherlands               8143
Australia                 6966
Poland                    5131
Ireland                   3965
Mexico                    3755
Portugal                  3723
Austria                   3457
Sweden                    3192
Finland                   2749
Norway                    2738
Bulgaria                  2631
Czech Republic            2629
Bolivia                   2449
New Zealand               2445
Argentina                 2144
Romania                   2025
Brazil                    1872
India                     1712
Russia                    1680
Thailand                  1604
Hungary                   1381
Croatia                   1247
Luxembourg                1061
Denmark 

In [56]:
# off_clean.groupby(
#     ['countries_en', 'parentcategory']).count()[['product_id']].reset_index().pivot(
#     index='countries_en', columns='parentcategory', values='product_id')

In [57]:
# off_clean[off_clean['ingredients_text_en']!=''].sample(100)['ingredients_text_en'].values

In [58]:
off[~off['product_id'].isin(off_clean['product_id'].unique())].shape

(265416, 22)

In [59]:
# off[~off['product_id'].isin(off_clean['product_id'].unique())].sample(100)['ingredients_text_en'].values

In [60]:
def translation_terms(text):
    
    text = text.replace(' ble ', ' wheat ')
    text = text.replace('soja ', ' soya ')
    text = text.replace(' leche ', ' milk ')
    text = text.replace(' ait ', ' milk ')
    text = text.replace(' oeuf ', ' egg ')
    text = text.replace(' euf ', ' egg ')
    text = text.replace(' oeufs ', ' eggs ')
    text = text.replace(' eufs ', ' eggs ')
    text = text.replace(' moutarde ', ' mustard ')
    text = text.replace(' sucre ', ' sugar ')
    text = text.replace(' beurre ', ' butter ')
    
    return text

In [61]:
off_clean['ingredients_text_en'] = off_clean['ingredients_text_en'].progress_apply(translation_terms)

  0%|          | 0/912480 [00:00<?, ?it/s]

#### standardize columns and merge datasets

In [62]:
# cleaning nutrient data
def nutrition_adjust_function(dat, nutrient_list):
    """
    Adjusts and standardizes nutrient data in a DataFrame.

    Parameters:
    - dat: pandas DataFrame containing nutrient information.
    - nutrient_list: list of nutrient column names to process.

    Returns:
    - Modified DataFrame with standardized nutrient values and units.
    """
    # Creating new columns to store data
    for nutrient in nutrient_list:
        dat[f"{nutrient}_value"] = np.nan
        dat[f"{nutrient}_unit"] = np.nan

    # Define units to extract
    unit_pattern = r'(mg|g|kcal|calorie|kj|j|kiloj)'

    # Define which nutrients are energy
    energy_nutrients = ['energy_per_100']

    for nutrient in nutrient_list:
        # Process each nutrient column
        value_col = f"{nutrient}_value"
        unit_col = f"{nutrient}_unit"

        # Ensure the nutrient column is string type for regex operations
        dat[nutrient] = dat[nutrient].astype(str)

        # Extract units using regex
        dat[unit_col] = dat[nutrient].str.extract(unit_pattern, flags=re.IGNORECASE, expand=False)
        dat[unit_col] = dat[unit_col].str.lower()

        # Extract numeric values using regex
        # Pattern to capture numbers with optional decimal separators and optional spaces
        number_pattern = r'(\d{1,4}\s*[.,]?\s*\d{0,3})'

        extracted_numbers = dat[nutrient].str.extract(number_pattern, flags=re.IGNORECASE, expand=False)
        extracted_numbers = extracted_numbers.str.replace(r'\s+', '', regex=True)  # Remove spaces
        extracted_numbers = extracted_numbers.str.replace(',', '.', regex=False)  # Replace comma with dot

        # Assign numeric values
        dat[value_col] = pd.to_numeric(extracted_numbers, errors='coerce')

        # Handle entries that are purely numeric (no units)
        # Detect if the entire string is a number
        purely_numeric_mask = dat[nutrient].str.strip().str.fullmatch(r'^\d+([.,]\d+)?$')

        # Assign values and units for purely numeric entries
        dat.loc[purely_numeric_mask, value_col] = pd.to_numeric(dat.loc[purely_numeric_mask, nutrient].str.replace(',', '.', regex=False), errors='coerce')
        # Assign default units: 'kj' for energy, 'g' for others
        default_unit = 'kj' if nutrient.lower() in [n.lower() for n in energy_nutrients] else 'g'
        dat.loc[purely_numeric_mask, unit_col] = default_unit

        # Adjusting units
        # Convert kcal to kj
        kcal_mask = dat[unit_col] == 'kcal'
        dat.loc[kcal_mask, value_col] = dat.loc[kcal_mask, value_col] * 4.184
        dat.loc[kcal_mask, unit_col] = 'kj'

        # Convert mg to g
        mg_mask = dat[unit_col] == 'mg'
        dat.loc[mg_mask, value_col] = dat.loc[mg_mask, value_col] / 1000
        dat.loc[mg_mask, unit_col] = 'g'

        # Define expected unit based on nutrient type
        expected_unit = 'kj' if nutrient.lower() in [n.lower() for n in energy_nutrients] else 'g'

        # Identify invalid units
        invalid_unit_mask = ~dat[unit_col].isin([expected_unit])

        # Set invalid entries to NaN
        dat.loc[invalid_unit_mask, value_col] = np.nan
        dat.loc[invalid_unit_mask, unit_col] = np.nan

        # Additionally, if the value is NaN, set unit to NaN
        value_nan_mask = dat[value_col].isna()
        dat.loc[value_nan_mask, unit_col] = np.nan

    return dat

In [63]:
# Define the list of nutrients
nutrient_list = ['energy_per_100', 'fat_per_100', 'saturates_per_100', 
                 'carbohydrate_per_100', 'fibre_per_100', 'protein_per_100', 
                 'sugar_per_100', 'salt_per_100']

# Apply the function
fdb = nutrition_adjust_function(fdb, nutrient_list)

In [64]:
# Apply the function
off_clean = nutrition_adjust_function(off_clean, nutrient_list)

In [65]:
off_clean['serving_size'] = off_clean['serving_size'].fillna('')
off_clean['serving_size_unit'] = 'g'
off_clean.loc[(~off_clean['serving_size'].str.contains('g'))
              & (off_clean['serving_size']!=''), 'serving_size_unit'] = 'ml'

In [66]:
off_clean = off_clean[['product_id', 'product_name_en', 'ingredients_text_en',
                       'countries_en', 'stores', 
                       'energy_per_100', 'fat_per_100', 'saturates_per_100', 
                       'carbohydrate_per_100', 'fibre_per_100', 'protein_per_100', 
                       'sugar_per_100', 'salt_per_100',
                       'energy_per_100_value', 'fat_per_100_value', 'saturates_per_100_value', 
                       'carbohydrate_per_100_value', 'fibre_per_100_value', 'protein_per_100_value', 
                       'sugar_per_100_value', 'salt_per_100_value',
                       'energy_per_100_unit', 'fat_per_100_unit', 'saturates_per_100_unit', 
                       'carbohydrate_per_100_unit', 'fibre_per_100_unit', 'protein_per_100_unit', 
                       'sugar_per_100_unit', 'salt_per_100_unit',
                       'serving_size', 'serving_quantity', 'serving_size_unit',
                       'parentcategory', 'mainfoodgroup','subfoodgroup']].rename(
    columns={'product_name_en': 'product_name', 
             'ingredients_text_en': 'ingredients_text',
             'countries_en': 'country',
             'stores': 'store',
             'serving_quantity': 'serving_size_value',
             'parentcategory': 'Department', 
             'mainfoodgroup': 'Aisle',
             'subfoodgroup': 'Shelf'})

In [67]:
off_clean['database'] = 'openfoodfacts'

In [68]:
off_clean.shape

(912480, 36)

In [69]:
fdb['country'] = 'United Kingdom'
fdb = fdb[['product_id', 'product_list_name', 'ingredients_text', 
           'country', 'store', 
           'energy_per_100', 'fat_per_100', 'saturates_per_100', 
           'carbohydrate_per_100', 'fibre_per_100', 'protein_per_100', 
           'sugar_per_100', 'salt_per_100',
           'energy_per_100_value', 'fat_per_100_value', 'saturates_per_100_value', 
           'carbohydrate_per_100_value', 'fibre_per_100_value', 'protein_per_100_value', 
           'sugar_per_100_value', 'salt_per_100_value',
           'energy_per_100_unit', 'fat_per_100_unit', 'saturates_per_100_unit', 
           'carbohydrate_per_100_unit', 'fibre_per_100_unit', 'protein_per_100_unit', 
           'sugar_per_100_unit', 'salt_per_100_unit',
           'serving_size', 'serving_size_value', 'serving_size_unit',
           'parentcategory_pred', 'mainfoodgroup_pred','subfoodgroup_pred']].rename(
    columns={'product_list_name': 'product_name',
             'parentcategory_pred': 'Department',
             'mainfoodgroup_pred': 'Aisle',
             'subfoodgroup_pred': 'Shelf'})

In [70]:
fdb.loc[(~fdb['serving_size_unit'].isin(['', 'g', 'ml']))
        & (fdb['serving_size_unit'].notnull()), 'serving_size_value'] = np.nan
fdb.loc[(~fdb['serving_size_unit'].isin(['', 'g', 'ml']))
        & (fdb['serving_size_unit'].notnull()), 'serving_size_unit'] = ''
fdb['database'] = 'fooddb'

In [71]:
fdb.shape

(242475, 36)

In [72]:
products_cats = pd.concat([off_clean, fdb], axis=0, ignore_index=True)

In [73]:
products_cats.shape

(1154955, 36)

In [74]:
products_cats['country_group'] = 'UK'
products_cats.loc[products_cats['country'].isin(['Germany', 'France', 'Austria', 'Belgium', 'Switzerland', 
                                                 'Netherlands', 'Luxembourg']), 'country_group'] = 'EUwest' 
products_cats.loc[products_cats['country'].isin(['Italy', 'Spain', 'Portugal']), 'country_group'] = 'EUsouth' 
products_cats.loc[products_cats['country'].isin(['Norway', 'Sweden', 'Finland', 'Denmark', 
                                                 'Lithuania', 'Latvia', 'Estonia']), 'country_group'] = 'Nordic' 
products_cats.loc[products_cats['country'].isin(['Poland', 'Czech Republic', 'Slovakia', 'Romania', 
                                                 'Hungary', 'Bulgaria', 'Slovenia', 'Croatia', 
                                                 'Serbia', 'Ukraine', 'Russia', 'New Caledonia']), 'country_group'] = 'EUeast' 
products_cats.loc[products_cats['country'].isin(['Greece', 'Turkey', 'Cyprus', 'Israel', 
                                                 'Lebanon', 'Egypt', 'Morocco', 'Tunisia', 
                                                 'Algeria', 'South Africa']), 'country_group'] = 'MedAf' 
products_cats.loc[products_cats['country'].isin(['India', 'United Arab Emirates', 'Saudi Arabia', 
                                                 'Qatar', 'Kuwait', 'Iraq', 'Réunion',
                                                 'Mexico', 'Argentina', 'Bolivia',  
                                                 'Venezuela', 'Peru', 'Chile', 'Ecuador', 'Brazil', 
                                                 'Cuba', 'Puerto Rico', 'Guadeloupe', 'Martinique', 
                                                 'French Polynesia', 'Panama', 'Costa Rica', 
                                                 'Colombia']), 'country_group'] = 'LatAmMeSAsia' 
products_cats.loc[products_cats['country'].isin(['Japan', 'South Korea', 'Thailand', 'Indonesia', 
                                                 'Philippines', 'Malaysia', 'Singapore', 'Hong Kong']), 'country_group'] = 'EAsia' 

In [75]:
# products_cats.sample(100)

In [76]:
products_cats[products_cats['ingredients_text']!=''].shape

(853710, 37)

In [77]:
products_cats[products_cats['ingredients_text']!='']['country_group'].value_counts()

UK              481908
EUwest          272284
EUsouth          44222
EUeast           18598
LatAmMeSAsia     17653
Nordic           10617
MedAf             4398
EAsia             4030
Name: country_group, dtype: int64

In [78]:
products_cats[products_cats['ingredients_text']!='']['country'].value_counts()

United Kingdom          237207
United States           222524
France                  178596
Germany                  50009
Spain                    25502
Switzerland              18095
Italy                    14997
Belgium                  12923
Canada                    8801
Netherlands               8143
Australia                 6966
Poland                    5131
Ireland                   3965
Mexico                    3755
Portugal                  3723
Austria                   3457
Sweden                    3192
Finland                   2749
Norway                    2738
Bulgaria                  2631
Czech Republic            2629
Bolivia                   2449
New Zealand               2445
Argentina                 2144
Romania                   2025
Brazil                    1872
India                     1712
Russia                    1680
Thailand                  1604
Hungary                   1381
Croatia                   1247
Luxembourg                1061
Denmark 

In [79]:
products_cats.to_csv('../../SFS/environmental_impacts/Products_dat/products_categories.csv', index=False)